In [11]:
linesCodeAnalyzer = None
with open('CodeJamCrawler/dataset_csvs/code_analyzer_out_sorted.csv', 'r') as f:
    linesCodeAnalyzer = f.readlines()
    
codeAnalyzerHeader = linesCodeAnalyzer[0]
linesCodeAnalyzer = linesCodeAnalyzer[1:]

linesCccc = None
with open('CodeJamCrawler/dataset_csvs/cccc_cleared.csv', 'r') as f:
    linesCccc = f.readlines()
    
ccccHeader = linesCccc[0]
linesCccc = linesCccc[1:]

print (linesCodeAnalyzer[0])
print (linesCccc[0])

0.0_14_16505_55.cpp,1,49,13,47,0,2,23.50,0.00,23.50,0.96,47,0,2

0.0_14_16505_55.csv;1;43;43.000;12;12.000;0;0.000;******;******;0;0.000;0;0.000;0;0.000;3



In [12]:
def get_file_name(line, splitter):
    entries = line.split(splitter)
    file_name = entries[0][0:-4]
    return file_name

In [13]:
print (get_file_name(linesCodeAnalyzer[0], ','))
print (get_file_name(linesCccc[0], ';'))

0.0_14_16505_55
0.0_14_16505_55


In [4]:
# removing all the entries that aren't existing in outputCleared
cpp_files_only = []
for i in range(len(linesCodeAnalyzer)):
    if i % 1000 == 0:
        print (i)
        
    other = linesCodeAnalyzer[i]
    file_name_other = get_file_name(other, ',')
    
    found = False
    for cleared in linesCccc:
        file_name_cleared = get_file_name(cleared, ';')
        if file_name_cleared == file_name_other:
            found = True
            break
    if not found:
        continue
    cpp_files_only.append(file_name_other)
    
print ("Number of cpp files:", len(cpp_files_only))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
('Number of cpp files:', 16521)


In [14]:
print (cpp_files_only[0])

NameError: name 'cpp_files_only' is not defined

In [15]:
print ('Code analyzer output:', codeAnalyzerHeader)
print ('CCCC analyzer output:', ccccHeader)

Code analyzer output: #File Name,Files,Lines,AVG Len,Code,Comments,White SP,Cd/Cm+WS,Cd/Cm,Cd/WS,% Code,Cd/File,Cm/File,WS/File,

CCCC analyzer output: #filename;number_of_modules;lines_of_code;lines_of_code_per_module;McCabes_cyclomatic_complexity;McCabes_cyclomatic_complexity_per_module;lines_of_comment;lines_of_comment_per_module;lines_of_code_per_line_of_comment;McCabes_cyclomatic_complexity_per_line_of_comment;IF4;IF4_per_module;IF4_visible;IF4_visible_per_module;IF4_concrete;IF4_concrete;rejected_lines_of_code



In [16]:
import numpy as np

def parse_cccc(cccc_line):
    name = get_file_name(cccc_line, ';')
    res = name.split('_')[-1]
    author = name.split('_')[0]
    features = []
    for num in cccc_line.split(';')[1:]:
        if num == '******' or num == '------':
            num = '0.0'
        try:
            features.append(float(num))
        except ValueError:
            print (num)
            break
    return author, features, res


In [17]:
def parse_code_analyzer(code_anal_line):
    name = get_file_name(code_anal_line, ',')
    res = name.split('_')[-1]
    author = name.split('_')[0]
    features = []
    for num in code_anal_line.split(',')[1:]:
        try:
            features.append(float(num))
        except ValueError:
            print (num)
            break
    return author, features, res

In [9]:
print ('cccc line vec', parse_cccc(linesCccc[0]))
print ('code line vec', parse_code_analyzer(linesCodeAnalyzer[0]))

('cccc line vec', ('0.0', [1.0, 43.0, 43.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0], '55'))
('code line vec', ('0.0', [1.0, 49.0, 13.0, 47.0, 0.0, 2.0, 23.5, 0.0, 23.5, 0.96, 47.0, 0.0, 2.0], '55'))


In [11]:
cpp_files_map = set()

for n in cpp_files_only:
    cpp_files_map.add(n.split('_')[0])

src_vec_map = {}

for cccc_line in linesCccc:
    name, features, res = parse_cccc(cccc_line)
    if name not in src_vec_map:
        src_vec_map[name] = [(features,res)]
    else:
        src_vec_map[name].append((features, res))

for code_anal_line in linesCodeAnalyzer:
    name, features, res = parse_code_analyzer(code_anal_line)
    if name in cpp_files_map:
        src_vec_map[name].append((features, res))

In [12]:
print (len(src_vec_map["4TShirt"]))
for v in src_vec_map["4TShirt"] :
    print (v)

54
([1.0, 74.0, 74.0, 16.0, 16.0, 47.0, 47.0, 1.574, 0.34, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '55')
([1.0, 74.0, 74.0, 16.0, 16.0, 47.0, 47.0, 1.574, 0.34, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '55')
([1.0, 85.0, 85.0, 18.0, 18.0, 47.0, 47.0, 1.809, 0.383, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0], '55')
([1.0, 47.0, 47.0, 10.0, 10.0, 46.0, 46.0, 1.022, 0.217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '55')
([1.0, 28.0, 28.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '55')
([1.0, 55.0, 55.0, 16.0, 16.0, 3.0, 3.0, 18.333, 5.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '6')
([1.0, 97.0, 97.0, 18.0, 18.0, 1.0, 1.0, 97.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0], '20')
([1.0, 97.0, 97.0, 18.0, 18.0, 1.0, 1.0, 97.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0], '20')
([1.0, 63.0, 63.0, 18.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0], '40')
([1.0, 63.0, 63.0, 18.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0], '40')
([1.0, 115.0, 115.0, 20.0, 20.0, 0.0, 0

In [18]:
print (len(src_vec_map))
csvs_count = 0
for k in src_vec_map:
    csvs_count += len(src_vec_map[k])
    
print ("Number of csvs: ", csvs_count)

NameError: name 'src_vec_map' is not defined

In [19]:
import random
def load_dataset(src_vec_map):
    dataset_list = []
    
    for k in src_vec_map:
        i = 0
        half_list = int(len(src_vec_map[k])/2)
        
        while i < half_list :
            label = src_vec_map[k][i][1]
            #print(label)
            l1 = np.asarray(src_vec_map[k][i][0])
            l2 = np.asarray(src_vec_map[k][i+half_list][0])
            #print (l1.shape, l2.shape)
            l = np.concatenate((l1, l2), axis=0)
            i += 1
            
            # pojma nemam zašto se ovo događa - goran
            if l.size != 29:
                continue
            dataset_list.append((l, (k, label)))
            
    dataset = np.array(dataset_list)
    dataset = dataset.reshape((dataset.shape[0], dataset.shape[1]))
    return dataset

In [20]:
dataset = load_dataset(src_vec_map)
print (dataset.shape, "NOTE : first dimension should be same as 18788/2\n\n")
print (dataset[0][1])

NameError: name 'src_vec_map' is not defined

<h1> TU SAM STAO    ,    JOSIP  </h1>

In [19]:
def get_author_features_and_scores(author_name, dataset):
    codes = []
    scores = []
    
    for item in dataset:
        if item[1][0] == author_name:
            codes.append(item[0])
            scores.append(item[1][1])
            
    return (codes, scores)  

def create_correct_np_array(data_list):
    arr = np.array([])
    arr = np.hstack((arr, np.array(data_list[0])))

    for i in range(1, len(data_list)):
        arr = np.vstack((arr, np.array(data_list[i])))
        
    return arr

In [20]:
# training and test authors, cca 70-30 omjer
author_names = list(src_vec_map.keys())

train_authors = author_names[0:1300]
test_authors = author_names[1300:]

print(len(train_authors))
print(len(test_authors))

X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for author in train_authors:
    codes, scores = get_author_features_and_scores(author, dataset)
    X_train_list.extend(codes)
    y_train_list.extend(scores)
    
for author in test_authors:
    codes, scores = get_author_features_and_scores(author, dataset)
    X_test_list.extend(codes)
    y_test_list.extend(scores)

1300
525


In [21]:
# iz nekog razloga svi ovi kodovi imaju više značajki nego što bi trebali
print("X_TRAIN\n")
for i in range(1, len(X_train_list)):
    if len (X_train_list[i]) != 29:
        print (i, "Broj značajki:", len (X_train_list[i]))

print("\nX_TEST\n")
for i in range(1, len(X_train_list)):
    if len (X_train_list[i]) != 29:
        print (i, "Broj značajki:", len (X_train_list[i]))

X_TRAIN


X_TEST



In [9]:
X_train = create_correct_np_array(X_train_list)
X_test = create_correct_np_array(X_test_list)
y_train = create_correct_np_array(y_train_list)
y_test = create_correct_np_array(y_test_list)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)


NameError: name 'create_correct_np_array' is not defined

<h1>TU SAM JA DOŠAO, GORAN</h1>

In [21]:
print (codeAnalyzerHeader.split(',')[1:-1], "\n")
print (ccccHeader.split(';')[1:], "\n")

labels = ccccHeader.split(';')[1:] + codeAnalyzerHeader.split(',')[1:-1]

print (len(labels))

['Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code', 'Cd/File', 'Cm/File', 'WS/File'] 

['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete', 'IF4_concrete', 'rejected_lines_of_code\n'] 

29


In [22]:
print X_train.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-22-2451cdbc6dcc>, line 1)

In [34]:
def enlist_kept_cols(X, labels):
    
    cnt_cols = X.shape[1]
    X_reduced = []
    #labels_reduced = []
    kept_cols = []
    
    for i in range(cnt_cols):
        col = X[:,i]
        notInside = True
        for j in range(len(X_reduced)):
            if (col == X_reduced[j]).all():
                notInside = False
        
        if notInside:
            kept_cols.append(i)
            #labels_reduced.append(labels[i])
            X_reduced.append(col)

    #return np.array(X_reduced).T, labels_reduced, kept_cols
    return kept_cols

#X, labels_reduced = remove_same_cols(X, labels)
# decision on which column will be kept or not is made on training part because it is bigger
kept_cols = enlist_kept_cols(X_train, labels)
print kept_cols
X_train_reduced = X_train[:, kept_cols]
print X_train_reduced.shape
X_test_reduced = X_test[:, kept_cols]
print X_test_reduced.shape
labels_reduced = [labels[i] for i in kept_cols]
print labels_reduced

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
(12658, 26)
(4686, 26)
['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete', 'IF4_concrete', 'rejected_lines_of_code\n', 'Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code']


In [23]:
import json
def create_json(author_names, dataset):
    data = {}
    data["column_descriptors"] = labels_reduced
    data["author_data"] = {}
    for author in author_names:
        feature_vecs, scores = get_author_features_and_scores(author, dataset)
        if len(scores) == 0:
            continue
        feat_vecs = [fv[kept_cols].tolist() for fv in feature_vecs]
        author_data = {}
        author_data["feature_vecs"] = feat_vecs
        author_data["scores"] = scores
        data["author_data"][author] = author_data
        
    with open('dataset.json', 'w') as outfile:
        json.dump(data, outfile)

#print get_author_features_and_scores(author_names[15], dataset)[1]
create_json(author_names, dataset)

NameError: name 'author_names' is not defined

In [24]:
import json
import numpy as np
def load_json():
    
    authors = []
    features = []
    scores = []
    
    with open('dataset.json') as data_file:
        data = json.load(data_file)
        labels = data["column_descriptors"]
        author_data = data["author_data"]
        
        for author in author_data.keys():
            authors.append(author)
            features.append(author_data[author]['feature_vecs'])
            scores_str = author_data[author]['scores']
            scores.append(np.asarray([int(score_) for score_ in scores_str]))
    
    # features to numpy array
    features = [np.asarray(feature) for feature in features]
    
    return authors, features, scores

In [25]:
authors, features, scores = load_json()
print (authors[0], features[0].shape, scores[0].shape)

Mavlarn (7, 26) (7,)


In [33]:
# training and test split
import random
cnt_train = 1095

print (cnt_train)

train_authors_indices = set()

while len(train_authors_indices) < cnt_train:
    train_authors_indices.add(random.randint(0, len(authors)))
    
test_authors_indices = set()

for i in range(len(authors)):
    if i not in train_authors_indices:
        test_authors_indices.add(i)

train_authors = [authors[i] for i in train_authors_indices]
        
test_authors = [authors[i] for i in test_authors_indices]

print(len(train_authors))
print(len(test_authors))

1095
1095
469


In [36]:
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for index in train_authors_indices:
    X_train_list.extend(features[index])
    y_train_list.extend(scores[index])
    
for index in test_authors_indices:
    X_test_list.extend(features[index])
    y_test_list.extend(scores[index])

X_train = np.asarray(X_train_list)
y_train = np.asarray(y_train_list)
X_test = np.asarray(X_test_list)
y_test = np.asarray(y_test_list)

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(12351, 26) (12351,) (4993, 26) (4993,)


<h1><b><i>Nikola</i></b></h1>
Momci, dosao sam do tuda. 
Odradena je pohrana i ucitavanje iz jsona, spremio sam samo one znacajke koje nisu duplikati. 
Odradio sam random uzorkovanje iz dataseta, a ne fiksno.
Train i test podskupovi su numpy arrayevi spremni za igru.

Slobodno mozete pokrenuti od celije gdje se nalazi naredba load_json().

In [37]:
labels_reduced = ['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete_', 'IF4_concrete', 'rejected_lines_of_code', 'Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code']
print (labels_reduced)

['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete_', 'IF4_concrete', 'rejected_lines_of_code', 'Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code']


In [40]:
import pandas as pd
#print 'labels', labels_reduced
df_before_feature_sel = pd.DataFrame(X_train)
#df_before_feature_sel.rename(columns=lambda x: x.strip(), inplace=True)
df_before_feature_sel.columns = [str(label).replace(' ', '_') for label in labels_reduced]
print (df_before_feature_sel.shape)

(12351, 26)


In [42]:
import seaborn as sns
import matplotlib.pyplot as plt

#df_before_feature_sel.columns = labels_reduced
sns.pairplot(df_before_feature_sel, size=3)
plt.show()

In [44]:
for i, label in enumerate(labels_reduced):
    print (i, label)

0 number_of_modules
1 lines_of_code
2 lines_of_code_per_module
3 McCabes_cyclomatic_complexity
4 McCabes_cyclomatic_complexity_per_module
5 lines_of_comment
6 lines_of_comment_per_module
7 lines_of_code_per_line_of_comment
8 McCabes_cyclomatic_complexity_per_line_of_comment
9 IF4
10 IF4_per_module
11 IF4_visible
12 IF4_visible_per_module
13 IF4_concrete_
14 IF4_concrete
15 rejected_lines_of_code
16 Files
17 Lines
18 AVG Len
19 Code
20 Comments
21 White SP
22 Cd/Cm+WS
23 Cd/Cm
24 Cd/WS
25 % Code


In [46]:
df_descriptions = df_before_feature_sel.describe()

for i in range(df_descriptions.shape[1]):
    #print labels_reduced[i]
    #print
    pass
print (df_descriptions.T)
#for i, feature in df_before_feature_sel.describe():
 #   print labels_reduced[i]
  #  print feature

                                                     count       mean  \
number_of_modules                                  12351.0   1.089466   
lines_of_code                                      12351.0  59.864869   
lines_of_code_per_module                           12351.0  56.177028   
McCabes_cyclomatic_complexity                      12351.0  16.606510   
McCabes_cyclomatic_complexity_per_module           12351.0  15.757898   
lines_of_comment                                   12351.0   4.187920   
lines_of_comment_per_module                        12351.0   3.657968   
lines_of_code_per_line_of_comment                  12351.0  12.601211   
McCabes_cyclomatic_complexity_per_line_of_comment  12351.0   3.433873   
IF4                                                12351.0   0.005182   
IF4_per_module                                     12351.0   0.000786   
IF4_visible                                        12351.0   0.002996   
IF4_visible_per_module                             

In [ ]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components='mle', svd_solver='full')
#pca.fit(X_train)

In [31]:
#pca.transform(X[0][:])

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ -7.40634652e+01,   8.77247004e+00,  -8.55460882e+00,
          6.70734217e-01,   2.82862328e+00,  -1.51354685e+00,
          4.32975137e+00,   9.17037916e-01,  -8.64835787e-01,
         -5.94518112e-01,  -4.54120767e-01,  -4.62262229e-01,
          3.23754372e+00,  -1.16529491e-02,   5.59282669e-01,
          2.57770958e-02,  -5.18105857e-02,  -5.55397692e-02,
          5.35324286e-02,   1.77726490e-03,   3.08480412e-02,
         -2.54754780e-02,   3.29777541e-05,   1.18303475e-05,
         -2.83296515e-06]])

In [48]:
# y is vector of labels
def create_labels(y):
    y_l = np.copy(y)
    for i in range(y.shape[0]):
        if y[i] >= 80: 
            y_l[i] = 5
        if y[i] >= 60 and y[i] < 80: 
            y_l[i] = 4
        if y[i] >= 40 and y[i] < 60:
            y_l[i] = 3
        if y[i] >= 20 and y[i] < 40:
            y_l[i] = 2
        if y[i] < 20: 
            y_l[i] = 1
    return y_l

In [50]:
y_train_labeled = create_labels(y_train)
y_test_labeled = create_labels(y_test)

In [66]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_features='log2')
rfc.fit(X_train, y_train)

print (rfc.score(X_test, y_test))

0.504105748047


In [54]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train) 

print (clf.score(X_test, y_test))

0.467854996996


In [56]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

print (clf.score(X_test, y_test))

0.0789104746645


In [67]:
from sklearn.model_selection import GridSearchCV
import multiprocessing
cores=multiprocessing.cpu_count()-2

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100, 1000]}
svc = SVC()
clf = GridSearchCV(svc, parameters, n_jobs=cores, verbose=2)
clf.fit(X_train, y_train)
print (clf.score(X_test, y_test))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=rbf .................................................
[CV] C=1, kernel=rbf .................................................
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total=  19.4s
[CV] C=10, kernel=linear .............................................
[CV] .................................. C=1, kernel=rbf, total=  19.3s
[CV] C=10, kernel=linear .............................................
[CV] .................................. C=1, kernel=rbf, total=  19.4s
[CV] C=10, kernel=linear .............................................


KeyboardInterrupt: 

In [58]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
print (gbc.score(X_test, y_test))

0.311436010415


In [60]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
print (abc.score(X_test, y_test))

0.0745043060284


In [61]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=30)
etc.fit(X_train, y_train)
print (etc.score(X_test, y_test))

0.504706589225


In [46]:
print X.shape, y.reshape((y.shape[0], 1)).shape

(19153, 26) (19153, 1)


In [47]:
#dataset = X#np.concatenate((X, y.T), axis=1)

dataset = np.concatenate((X,y[:,None]),axis=1)
#print X.shape[0] == y.shape[0]

In [49]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df = pd.DataFrame(dataset)
#sns.pairplot(df)
#plt.show()